UI

In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv("UI.csv")

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(","))
action_features = vectorizer.fit_transform(df['action']).toarray()

action_df = pd.DataFrame(action_features, columns=vectorizer.get_feature_names_out())

df_final = pd.concat([df[['age_days', 'time_takes']], action_df], axis=1)

X = df_final
y = df['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

c:\Users\User\Documents\RoadTo15k\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       1.00      0.88      0.93         8

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



c:\Users\User\Documents\RoadTo15k\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:14:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [7]:
import pickle

with open("UI.pkl", "wb") as file:
    pickle.dump(model, file)

In [9]:
with open("UI.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

with open("UI.pkl", "rb") as file:
    model = pickle.load(file)

df = pd.read_csv("UI.csv")  

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(","))
vectorizer.fit(df["action"])  

new_data = pd.DataFrame({
    "age_days": [200],  
    "action": ["profile,deposit,trade in"],  
    "time_takes": [38]
})

action_features = vectorizer.transform(new_data["action"]).toarray()
action_df = pd.DataFrame(action_features, columns=vectorizer.get_feature_names_out())

new_data_final = pd.concat([new_data[['age_days', 'time_takes']], action_df], axis=1)

prediction = model.predict(new_data_final)
print("Predicted Label:", prediction[0])


Predicted Label: 0


c:\Users\User\Documents\RoadTo15k\.venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fraud Prevention

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 读取数据
df = pd.read_csv("Updated_Trading_Dataset.csv")

# 处理时间列（转换为时间戳）
df["TradeIn_DateTime"] = pd.to_datetime(df["TradeIn_DateTime"]).astype(int) // 10**9
df["TradeOut_DateTime"] = pd.to_datetime(df["TradeOut_DateTime"]).astype(int) // 10**9

# 处理类别列（转换为 category 类型）
categorical_cols = ["DepositBank", "WithdrawalBank"]
for col in categorical_cols:
    df[col] = df[col].astype("category").cat.codes  # 用整数编码类别

# 选择特征和标签
X = df.drop(columns=["target_column"])  # 你的目标列
y = df["target_column"]

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练 XGBoost
model = xgb.XGBClassifier(eval_metric="logloss", enable_categorical=True)
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)

# 输出结果
print(predictions)


ValueError: time data "31/1/2025 17:28" doesn't match format "%m/%d/%Y %H:%M", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.